# Self-Improving Agents with TextGrad and TogetherAI

Author: [Federico Bianchi](https://federicobianchi.io/)

## Short Summary

In this tutorial, we'll explore how to create self-improving AI systems using TextGrad and TogetherAI models. TextGrad is a powerful framework that enables automatic "differentiation" via text, allowing language models to improve themselves through textual feedback. Thanks to TogetherAI, we will be able to use TextGrad to optimize prompt from open-source models.

A self-optimization framework for agents looks like this:


<div align="center"><img src="https://www.agentrecipes.com/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fevaluator-optimizer.32896758.png&w=3840&q=75&dpl=dpl_Ek3R4Pxv85QsXYQja11CAeaqGhMf" width = 1000></div>

<div align="center"><i>Image from <a href="https://www.agentrecipes.com">Agent Recipes</a></i></div>

An LLM generate some content and then another LLM critisizes that comments and provide feedback. The feedback is used to improve the original answers. Improvement can be done N-times or until the "evaluator" is satisifed with the answer.

There is another tutorial on looping agents from scratch form [Zain Hasan]() that might be an [interesting read]()


## Goal

In this tutorial, we'll explore several practical applications of TextGrad with TogetherAI models. We'll progress through three key examples of increasing complexity:

1. Test-Time Optimization - A simple warm-up to introduce the core concepts
2. Prompt Optimization - A more sophisticated approach to enhance model performance
3. Self-Improving Coding Agent - An advanced implementation with built-in optimization capabilities

By the end of this tutorial, you'll be able to:
* Understand the versatile applications of TextGrad for LLM optimization
* Leverage TogetherAI's powerful models for test-time optimization
* Implement your own optimization pipelines for various use cases
* Apply these techniques to create more capable and efficient AI systems

Each section builds kind of build on the abstractions of the previous one, but you should be able to skip without losing too much context.

# What is TextGrad?

[TextGrad](https://github.com/zou-group/textgrad) is a recent framework for the end-to-end optimization of language models prompts thorugh text feedback. What this basically means is that TextGrad will allow you to optimize language models' prompts and solutions automatically.

Key features of TextGrad:
- Provides a PyTorch-inspired API for defining and optimizing text-based variables
- Enables backpropagation of textual feedback to improve individual components
- Works with a variety of tasks without requiring framework modifications
- Supports optimization of diverse elements from prompts to code snippets (anything that is text)
- Supports feedback for multimodal models

A few different use-cases can be implemented in TextGrad, but the main two are:

1) Prompt Optimization

2) Test-Time Optimization

* TextGrad will give us easy to use abstractions to build an optimization layer on top of agents or llm calls.
* TogetherAI will give us the models to do this in an effective way!

## Which Kind of "Optimization" is TextGrad doing?

TextGrad optimizes language models using textual feedback. At the most basic level, a language model provides feedback to LLMs components (solutions, prompts) and identifies improvement opportunities. At the more sophisticated level, TextGrad offers primitives for end-to-end optimization of complex Language Model pipelines composed of multiple steps. By backpropagating feedback across different levels of LLM chains, TextGrad enables comprehensive optimization that significantly improves results.

This approach allows for optimization of elements that traditional gradient-based methods cannot handle, such as discrete text structures and reasoning patterns.

Fun fact: The optimization loop in TextGrad is inspired by [Karpathy's micrograd](https://github.com/karpathy/micrograd). We literally follow the same signatures to implement an AutoGrad system for Text.

Here you can find an image comparing TextGrad to torch.

<img src="https://github.com/zou-group/textgrad/raw/main/assets/analogy.png"/>



In [ ]:
import random
import time
# note that as of this time, litellm engines are still experimental
from textgrad.engine_experimental.litellm import LiteLLMEngine
import textgrad as tg
import textgrad as tg
from textgrad.tasks import load_task
from dotenv import load_dotenv
from textgrad.loss import MultiFieldTokenParsedEvaluation


load_dotenv() # or make sure you have the TOGETHER_API_KEY env variable set, this is the only key we need for this tutorial

# a simple support function to print text with some newlines
def wrap_text(text, max_width=100):
    if not text:
        return ""
        
    result = []
    current_line = []
    current_length = 0
    
    # Split by existing newlines first to preserve intentional line breaks
    for line in text.split('\n'):
        words = line.split()
        current_length = 0
        current_line = []
        
        for word in words:
            # If adding this word would exceed max_width
            if current_length + len(word) + (1 if current_line else 0) > max_width:
                # Add the current line to result and start a new line
                result.append(' '.join(current_line))
                current_line = [word]
                current_length = len(word)
            else:
                # Add word to current line
                if current_line:  # Add space before word if not first word
                    current_length += 1  # Account for space
                current_line.append(word)
                current_length += len(word)
        
        # Add the last line from this original line
        if current_line:
            result.append(' '.join(current_line))
    
    return '\n'.join(result)

### How to use TogetherAI with TextGrad

Thanks to LiteLLM integration, we can simply use TogetherAI models through their API. LiteLLM handles all the authentication and API calls behind the scenes (assuming you have loaded the env variables), making it very easy to use different models. In this case, we're using Meta's Llama 3 model hosted on TogetherAI's platform.

In [149]:
generation = LiteLLMEngine("together_ai/meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo", cache=True).generate(content="hello, what's 3+4", system_prompt="you are an assistant")
generation

'Hello. 3 + 4 is 7.'

# Warming Up Example: Test-Time Improvement

So what can TextGrad do for us in practice? TextGrad can provide a layer of optimization on top of your agents, by checking their prompts and correcting them in case there is the need to do so. Let's start with an easy example.

Let's assume our LLM has just generated the following:

In [70]:
initial_solution = """To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:
x = (-b ± √(b^2 - 4ac)) / 2a
a = 3, b = -7, c = 2
x = (7 ± √((-7)^2 + 4(3)(2))) / 6
x = (7 ± √73) / 6
The solutions are:
x1 = (7 + √73)
x2 = (7 - √73)"""

The solution to this math problem is wrong (you can check it or you can actually give this in input to an LLM which will tell you where the problem is (which is also basicall what textgrad is doing behind the secens!))

Now, let's use TextGrad to improve this solution.

In [81]:
engine = LiteLLMEngine("together_ai/meta-llama/Llama-3.3-70B-Instruct-Turbo", cache=True)

# here we set the engine that will be used to compute the gradients and update the solution
tg.set_backward_engine(engine, override=True) 


In [89]:
# TextGrad defines variables that will be optimized, this is similar to Torch Tensors.

solution = tg.Variable(initial_solution,
                       requires_grad=True,
                       role_description="solution to the math question")

# This is the system prompt that will guide the loss function.
loss_system_prompt = tg.Variable("""You will evaluate a solution to a math problem. There is no reason to solve it yourself, do not give a solution to the problem, only identify errors.""",
                                 requires_grad=False,
                                 role_description="system prompt")

# This is the loss function, it will be used to compute the loss and the gradients.
loss_fn = tg.TextLoss(loss_system_prompt)

# This is the optimizer, it will be used to update the solution. TGD stands for Textual Gradient Descent, which is an analogy to the classic gradient descent but entirely based on text.
optimizer = tg.TGD([solution])

In [90]:
loss = loss_fn(solution)
print(loss.value)


The error in the solution is in the calculation of the discriminant (the expression inside the square root) and the final expressions for x1 and x2.

The correct calculation for the discriminant is:
(-7)^2 - 4(3)(2) = 49 - 24 = 25

So, the correct expression for x is:
x = (7 ± √25) / 6
x = (7 ± 5) / 6

And the correct solutions are:
x1 = (7 + 5) / 6 = 12 / 6 = 2
x2 = (7 - 5) / 6 = 2 / 6 = 1/3

The given solutions x1 = (7 + √73) and x2 = (7 - √73) are incorrect because they do not have the division by 6, and the discriminant is incorrectly calculated as √73 instead of √25.


In [91]:
loss.backward()
optimizer.step()

print(solution.value)

To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula: 
x = (-b ± √(b^2 - 4ac)) / 2a 
a = 3, b = -7, c = 2 
x = (7 ± √((-7)^2 - 4(3)(2))) / 6 
x = (7 ± √(49 - 24)) / 6 
x = (7 ± √25) / 6 
x = (7 ± 5) / 6 
The solutions are: 
x1 = (7 + 5) / 6 = 12 / 6 = 2 
x2 = (7 - 5) / 6 = 2 / 6 = 1/3


That was the correct solution! You can replace the solution with anything you want to optimize. And you can optimize it multiple times in a for loop (same as you would do with Torch)

There are quite a few things that TextGrad can do, mimicing Torch. For example, we can add textual contraints to the optimization process.

In [150]:
solution = tg.Variable(initial_solution,
                       requires_grad=True,
                       role_description="solution to the math question")

loss_system_prompt = tg.Variable("""You will evaluate a solution to a math problem. There is no reason to solve it yourself, do not give a solution to the problem, only identify errors.""",
                                 requires_grad=False,
                                 role_description="system prompt")
                              
loss_fn = tg.TextLoss(loss_system_prompt)
optimizer = tg.TGD([solution], constraints=["Make sure that you use latex"])


loss.backward()
optimizer.step()

print(wrap_text(solution.value))

To solve the equation $3x^2 - 7x + 2 = 0$, we can use the quadratic formula: $x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}$. In this case, $a = 3$, $b = -7$, and $c = 2$. Plugging these values into the formula, we get: $x = \frac{-(-7) \pm \sqrt{(-7)^2 - 4(3)(2)}}{2(3)} = \frac{7 \pm \sqrt{49 - 24}}{6} = \frac{7 \pm \sqrt{25}}{6}$. Therefore, the solutions are: $x_1 = \frac{7 + 5}{6} = \frac{12}{6} = 2$ and $x_2 = \frac{7 - 5}{6} = \frac{2}{6} = \frac{1}{3}$. Alternatively, the solutions can be written as: $x_1 = \frac{7 + \sqrt{25}}{6}$ and $x_2 = \frac{7 - \sqrt{25}}{6}$, but since $\sqrt{25} = 5$, the first representation is more simplified. Note that the original solutions $x1 = (7 + \sqrt{73})$ and $x2 = (7 - \sqrt{73})$ are incorrect because they do not satisfy the given equation $3x^2 - 7x + 2 = 0$.


## System Prompt Optimization

In this section, we'll explore how TextGrad enables prompt optimization - a powerful technique to enhance model performance without changing the model itself.


We'll demonstrate how to optimize the system prompt for a smaller model (Llama-3.2-3B) using feedback from a much larger model (70B parameters). This approach leverages the capabilities of the larger model to guide and improve the smaller one, making it more efficient and cost-effective.

For our example, we'll tackle a challenge from the Big Bench Hard (BBH) dataset - specifically the object counting task. This task presents the model with a list of various objects and tests its ability to accurately count specific items, a seemingly simple task that smaller models often struggle with.


In [303]:
# we define our target small model
llm_engine_small = LiteLLMEngine(model_string="together_ai/meta-llama/Llama-3.2-3B-Instruct-Turbo")

_, val_set, _, eval_fn = load_task("BBH_object_counting", llm_engine_small)
question_str, answer_str = val_set[2]

question = tg.Variable(question_str, role_description="question asked to the LLM", requires_grad=False)
answer = tg.Variable(str(answer_str), role_description="correct answer to the question", requires_grad=False)


In [304]:
# this is the system prompt we want to optimize, we will start with a simple CoT prompt.
system_prompt = tg.Variable("You are a concise LLM. Think step by step.",
                            requires_grad=True,
                            role_description="system prompt to guide the LLM's reasoning strategy for accurate responses")

model = tg.BlackboxLLM(llm_engine_small, system_prompt=system_prompt)
optimizer = tg.TGD(parameters=list(model.parameters()))

prediction = model(question)

In [305]:
print(question)
print(answer)
print()
print(prediction)

I have a stalk of celery, two drums, two onions, a carrot, an accordion, a yam, a cabbage, a lettuce head, a potato, and a head of broccoli. How many vegetables do I have?
9

To find the number of vegetables, let's identify the vegetables in the list:

1. Celery
2. Onion
3. Carrot
4. Cabbage
5. Lettuce
6. Broccoli
7. Potato

There are 7 vegetables in the list.


### Loss

For our prompt optimization, we'll use one of TextGrad's powerful built-in loss functions.

This specialized function acts like a smart evaluator, comparing our variables (like questions and answers) and providing structured feedback. Each variable gets assigned a specific "role" so the system understands what it's looking at - think of it as giving clear job descriptions to each piece of data.

One important tip: pay attention to the order of these roles! The sequence matters significantly for how the relationships between variables are interpreted. Get this right, and TextGrad will generate precisely the feedback needed to refine your prompts.

In [306]:
evaluation_instruction = "Below is a question from a question-answering task, the ground truth answer, and reasoning with the final prediction. Is the final prediction correct, i.e. the same as the ground truth answer? Say only 1 (yes) or 0 (no). Return your response within <ACCURACY> </ACCURACY> tags. e.g.<ACCURACY> 0 </ACCURACY> or <ACCURACY> 1 </ACCURACY>"
eval_instruction = tg.Variable(evaluation_instruction, requires_grad=False, role_description="evaluation instruction for the task")

role_descriptions = [
    "Question for the task",
    "Ground truth answer",
    "Reasoning and prediction from the language model"
]

loss_fn = MultiFieldTokenParsedEvaluation(
    eval_instruction,
    role_descriptions=role_descriptions,
    parse_tags=["<ACCURACY>", "</ACCURACY>"]
)

In [307]:
# ok let's see the loss
optimizer.zero_grad()

loss = loss_fn([question, answer, prediction])
print(loss.value)


<ACCURACY> 0 </ACCURACY>


ok, the loss function has just said that our result is not correct.

In [308]:
loss.backward()

optimizer.step()

prediction = model(question)


let's now check the new prediction

In [309]:
print(prediction.value)

To identify the vegetables, I will consider the characteristics and common categorizations of each item.

1. Celery - a stalk of celery is a vegetable.
2. Drums - not a vegetable, but a musical instrument.
3. Onions - a vegetable.
4. Onions - another instance of onions, so it's a vegetable.
5. Carrot - a vegetable.
6. Accordion - not a vegetable, but a musical instrument.
7. Yam - a vegetable.
8. Cabbage - a vegetable.
9. Lettuce head - a vegetable.
10. Potato - a vegetable.
11. Broccoli - a vegetable.

Counting each instance of the same item, I have:

- Celery: 1
- Onions: 2
- Carrot: 1
- Yam: 1
- Cabbage: 1
- Lettuce head: 1
- Potato: 1
- Broccoli: 1

In total, I have 9 vegetables.


What is the system prompt that helped us getting the correct result?

In [312]:
print(wrap_text(model.system_prompt.value))

You are a concise LLM. When approaching this task, ensure that you consider all items in the list,
including multiple instances of the same item. Think critically about what constitutes a vegetable,
and use this knowledge to inform your identification and counting. If an item could be classified in
multiple ways, consider its most relevant characteristics or categorizations. After generating your
list of identified vegetables, cross-check it against the original list of items to ensure that no
vegetables were missed. Evaluate your own performance and identify areas for improvement. Analyze
the list of items and identify any patterns or relationships that could inform your identification
and counting of vegetables. When encountering multiple instances of the same item, ensure that you
count each instance separately.


This is a very specific use-case and in general, you would prefer to optimize on multiple examples to avoid ending up overfitting on single
examples. There is a full tutorial about this here.

# Agents Writing Code

Let's now look at the full extent of TextGrad flexibility. We will write a loss function for test-time optimization. This is one of the most advanced usages of TextGrad so a few pieces ended up being involved in this process. We'll see how we can optimize code generation directly.

One of the most popular usecases nowadays is having agents writing code. However, most often these agents do not give you optimized outputs. 

We will simplify part of the interaction with the language model to make the example easier to follow.


### SideNote
Note that what we will be doing will happen completely in an unsupervised way. TextGrad supports supervised optimization (e.g., you get an llm output, you evaluate that using an external function (e.g., accuracy, and you tell the LLM that result and it will be used to optimize, you can find an example of this in another tutorial))


### Some Evaluation Code (Plase open, read carefully)

The code below contains a safety mechanism that prevents automatic execution. Before running,
 you'll need to review the code and uncomment an exception. For best practices, we recommend
running this in a sandboxed environment like Google Colab.

In [143]:
# We'll use below utilities to run a python function.
from IPython.core.interactiveshell import InteractiveShell

def run_function_in_interpreter(func_code):
    #raise Exception("This function will run the code returned by GPT-4o. Remove this if you'd like to run the code!")
    interpreter = InteractiveShell.instance()
    
    interpreter.run_cell(func_code, store_history=False, silent=True)
    
    func_name = func_code.split("def ")[1].split("(")[0].strip()
    func = interpreter.user_ns[func_name]
    
    return func


def test_longest_increasing_subsequence(fn):
    nums = [10, 22, 9, 33, 21, 50, 41, 60]
    assert fn(nums) == 5

    nums = [7, 2, 1, 3, 8, 4, 9, 6, 5]
    assert fn(nums) == 4

    nums = [5, 4, 3, 2, 1]
    assert fn(nums) == 1

    nums = [1, 2, 3, 4, 5]
    assert fn(nums) == 5

    nums = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]
    assert fn(nums) == 4

    nums = [10, 9, 2, 5, 3, 7, 101, 18]
    assert fn(nums) == 4

    nums = [0, 8, 4, 12, 2, 10, 6, 14, 1, 9, 5, 13, 3, 11, 7, 15]
    assert fn(nums) == 6

    nums = [7, 7, 7, 7, 7, 7, 7]
    assert fn(nums) == 1

    nums = [20, 25, 47, 35, 56, 68, 98, 101, 212, 301, 415, 500]
    assert fn(nums) == 11

    nums = [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
    assert fn(nums) == 1

    print("✅ All test cases passed!")

# Generate a random test case
def generate_random_test_case(size, min_value, max_value):
    return [random.randint(min_value, max_value) for _ in range(size)]

# Test the function with a random test case
size = 10000  # Adjust the size as needed
min_value = 1
max_value = 10000

nums = generate_random_test_case(size, min_value, max_value)

# When evaluating the code, we will run it through this function
# this will print the results and the runtime
def test_lis_implementation(code):
    longest_increasing_subsequence = run_function_in_interpreter(code)

    start_time = time.time()
    lis = longest_increasing_subsequence(nums)
    end_time = time.time()

    print(f"Test Case Size: {size}")
    print(f"Longest Increasing Subsequence Length: {lis}")
    print(f"Runtime: {end_time - start_time:.5f} seconds")

    # Test for all test cases
    test_longest_increasing_subsequence(longest_increasing_subsequence)

## Coding Problems

In [132]:
problem_text = """Longest Increasing Subsequence (LIS)

Problem Statement:
Given a sequence of integers, find the length of the longest subsequence that is strictly increasing. 
A subsequence is a sequence that can be derived 
from another sequence by deleting some or no elements without changing the order of the remaining elements.

Input:
The input consists of a list of integers representing the sequence.

Output:
The output should be an integer representing the length of the longest increasing subsequence.

Only return the function implementation using the following format:

```python
def longest_increasing_subsequence(nums):
[... your code ...]
```
"""

problem = tg.Variable(problem_text, requires_grad=False, role_description="problem statement")

In [133]:
# We are using a pretty big model here, so one would expect the code to be good!
engine = LiteLLMEngine("together_ai/meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo", cache=True)
tg.set_backward_engine(engine, override=True)

model = tg.BlackboxLLM(engine)

answer = model(problem)

first_answer = answer.value.split("```python")[1].split("```")[0]

print(first_answer)


Is this answer good?

In [144]:
test_lis_implementation(first_answer)

Test Case Size: 10000
Longest Increasing Subsequence Length: 195
Runtime: 2.34490 seconds
All test cases passed!


It is! but maybe we can do better? this looks rather slow? Let's implement an entire textgrad pipeline this time.

In [145]:
# Code is the variable of interest we want to optimize -- so requires_grad=True
code = tg.Variable(value=first_answer,
                 requires_grad=True,
                 role_description="code instance to optimize")

# We are not interested in optimizing the problem -- so requires_grad=False
problem = tg.Variable(problem_text, 
                    requires_grad=False, 
                    role_description="the coding problem statement")

# Let TGD know to update code!
# Like Torch!
optimizer = tg.TGD(parameters=[code])

### Custom Losses In TextGrad

In [146]:
# The system prompt that will guide the behavior of the loss function.
loss_system_prompt = "You are a smart language model that evaluates code snippets. You do not solve problems or propose new code snippets, only evaluate existing solutions critically and strong feedback."
loss_system_prompt = tg.Variable(loss_system_prompt, requires_grad=False, role_description="system prompt to the loss function")

# The instruction that will be the prefix
instruction = """Think about the problem and the code snippet. What is the runtime complexity?"""

# The format string and setting up the call
format_string = "{instruction}\nProblem: {{problem}}\nCurrent Code: {{code}}"
format_string = format_string.format(instruction=instruction)

fields = {"problem": None, "code": None}
formatted_llm_call = tg.autograd.FormattedLLMCall(engine=engine,
                                                  format_string=format_string,
                                                  fields=fields,
                                                  system_prompt=loss_system_prompt)

# Finally, the loss function
def loss_fn(problem: tg.Variable, code: tg.Variable) -> tg.Variable:
    inputs = {"problem": problem, "code": code}
    
    return formatted_llm_call(inputs=inputs,
                              response_role_description=f"evaluation of the {code.get_role_description()}")



We are ready to see some magic!

In [147]:
loss = loss_fn(problem, code)
print(loss.value)


loss.backward()
print(code.gradients)

optimizer.step()

The runtime complexity of the given code snippet is O(n^2), where n is the length of the input list `nums`.

Here's why:

- The outer loop iterates over the list `nums`, which takes O(n) time.
- The inner loop also iterates over the list `nums` up to the current index `i`, which takes O(n) time in the worst case.
- Since the inner loop is nested within the outer loop, the total time complexity is O(n) * O(n) = O(n^2).

This is not the most efficient solution for the Longest Increasing Subsequence problem, especially for large inputs. A more efficient solution would use a dynamic programming approach with a binary search, which can achieve a time complexity of O(n log n). However, the given code snippet is a straightforward and simple implementation of the dynamic programming approach, and it is correct in terms of logic and output. 

One minor improvement could be to add some error checking to handle the case where the input is not a list or contains non-integer values. However, this w

In [148]:
test_lis_implementation(code.value)

Test Case Size: 10000
Longest Increasing Subsequence Length: 195
Runtime: 0.00660 seconds
All test cases passed!


So FAST!

# Other Possibilies

TextGrad can be applied to many different use-cases. Some examples are described in the paper:

* Long Chain Optimization
* System Prompt Optimization